In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [2]:
os.getcwd()

'C:\\Users\\jwats\\Documents\\CodeandData\\Jupyter Notebooks'

### Loading and Cleaning DataFrame

In [3]:
df=pd.read_csv('exp_results/GROUP4-FINAL exp2c-table.csv',skiprows=6).rename(columns={})
df=df.copy().rename(columns={ 'japanese-resupply-rate':'j-resupp',
                             'japanese_supplies':'j-supp', 
                             'britain-casualties-hunger':'b-hung',
                             'britain-casualties-combat':'b-comb',
                             'japan-casualties-hunger':'j-hung',
                             'japan-casualties-combat':'j-comb', 
                             'control-imphal':'imp',
                             'control-kohima':'koh'})
df1=df[['ticks','j-supp','b-hung','b-comb','j-hung','j-comb', 'imp', 'koh']].copy()

In [4]:
df1=df[['ticks','j-supp','j-resupp','b-hung','b-comb','j-hung','j-comb', 'imp', 'koh']].copy()
df1.loc[df1['imp']=='Britain','imp']=0.0
df1.loc[df1['imp']=='Japan','imp']=1.0
df1.loc[df1['koh']=='Britain','koh']=0.0
df1.loc[df1['koh']=='Japan','koh']=1.0
df1['koh']=df1['koh'].astype(int)
df1['imp']=df1['imp'].astype(int)
df1['j-tot']=df1['j-comb']+df1['j-hung']
df1['b-tot']=df1['b-comb']+df1['b-hung']
df1['j-per']=df1['j-comb']/df1['j-tot']
df1['b-per']=df1['b-comb']/df1['b-tot']

### Summary Stats and Regression Analysis (for tables)

In [5]:
#Summary stats
exp2c=df1.groupby(['j-resupp','j-supp'],as_index=False).agg({'j-tot' : 'mean', 
                                               'b-tot' : 'mean',
                                               'imp':'sum',
                                               'koh':'sum',
                                               'ticks':'mean',
                                               'j-per':'mean',
                                               'b-per':'mean'})
a=exp2c.rename(columns={'j-resupp':'Resupply Rate',
                        'j-supp':'Initial supplies', 
                        'j-tot':'Axis Casualties', 
                        'b-tot':'Ally Casualties', 
                        'imp':'Imphal Captured', 
                        'koh':'Kohima Captured', 
                        'ticks':'Days', 
                        'j-per':'% Combat'})
b=a[['Resupply Rate','Initial supplies', 'Axis Casualties', 'Ally Casualties',
       'Imphal Captured', 'Kohima Captured', 'Days', '% Combat']]
b['% Combat']=b['% Combat']*100
c=b.round().astype(int)
c.sort_values('Imphal Captured',ascending=False)

C:\Users\jwats\AppData\Local\Temp\ipykernel_14516\407502511.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['% Combat']=b['% Combat']*100


,Resupply Rate,Initial supplies,Axis Casualties,Ally Casualties,Imphal Captured,Kohima Captured,Days,% Combat
16,10,75,75745,116872,100,100,253,100
14,10,25,75542,115739,100,100,239,100
17,10,100,75241,117135,99,100,253,100
10,8,75,75892,116961,99,100,254,100
11,8,100,75942,116957,99,100,254,100
15,10,50,75437,117078,97,100,253,100
13,10,10,78477,114735,95,99,240,100
5,6,100,77896,116356,92,100,256,100
9,8,50,80060,115116,73,100,262,100
4,6,75,81162,113892,54,100,262,100


In [6]:
nump_var=df1[['j-supp','j-resupp']].to_numpy()

In [7]:
### Regression 1: casualties vs supplies
import statsmodels.api as sm

#define response variable
y = df1['j-tot']

#define predictor variables
x = nump_var

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.params[1], model.params[2])
print(model.bse[1],model.bse[2])
print(model.fvalue)
print(model.pvalues[1],model.pvalues[2])
print(model.rsquared)

-8.055966300366359 -17.739166666890302
3.196382225108304 69.58710814129613
3.2085461389494236
0.011810186810275253 0.7988138249752197
0.0035582962506989224


In [8]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  j-tot   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     3.209
Date:                Sun, 23 Apr 2023   Prob (F-statistic):             0.0406
Time:                        11:55:37   Log-Likelihood:                -17818.
No. Observations:                1800   AIC:                         3.564e+04
Df Residuals:                    1797   BIC:                         3.566e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.855e+04    584.816    134.309      0.000    7.74e+04    7.97e+04
x1            -8.0560      3.196     -2.520      0.012     -14.325      -1.787
x2           -17.7392     69.587     -0.255      0.799    -154.219     118.741
==============================================================================
Omnibus:                      287.295   Durbin-Watson:                   0.320
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              471.705
Skew:                          -1.052   Prob(JB):                    3.72e-103
Kurtosis:                       4.364   Cond. No.                         292.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
### Regression 2: days vs supplies
import statsmodels.api as sm

#define response variable
y = df1['ticks']

#define predictor variables
x = nump_var

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.params[1], model.params[2])
print(model.bse[1],model.bse[2])
print(model.fvalue)
print(model.pvalues[1],model.pvalues[2])
print(model.rsquared)

0.6439589743589746 7.266666666665518
0.021218388139008122 0.46193670406959036
584.2640723839512
1.0906726350701743e-163 2.4429142265469657e-52
0.39403711167926125


In [10]:
### Regression 3: days vs casualties
import statsmodels.api as sm

#define response variable
y = df1['imp']

#define predictor variables
x = nump_var

#add constant to predictor variables
x = sm.add_constant(x)


#view model summary
model = sm.Logit(y, x).fit()

#view model summary
print(model.summary())
print(model.summary2())

Optimization terminated successfully.
         Current function value: 0.174580
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                    imp   No. Observations:                 1800
Model:                          Logit   Df Residuals:                     1797
Method:                           MLE   Df Model:                            2
Date:                Sun, 23 Apr 2023   Pseudo R-squ.:                  0.7481
Time:                        11:55:37   Log-Likelihood:                -314.24
converged:                       True   LL-Null:                       -1247.5
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -29.2014      1.908    -15.303      0.000     -32.941     -25.461
x1             0.1594      0

In [11]:
### Regression 4: capturing imphal vs supplies
import statsmodels.api as sm

#define response variable
y = df1['koh']

#define predictor variables
x = nump_var

#add constant to predictor variables
x = sm.add_constant(x)

model = sm.Logit(y, x).fit()

#view model summary
print(model.summary())
print(model.summary2())

Optimization terminated successfully.
         Current function value: 0.112763
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                    koh   No. Observations:                 1800
Model:                          Logit   Df Residuals:                     1797
Method:                           MLE   Df Model:                            2
Date:                Sun, 23 Apr 2023   Pseudo R-squ.:                  0.8123
Time:                        11:55:37   Log-Likelihood:                -202.97
converged:                       True   LL-Null:                       -1081.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -23.6905      1.765    -13.425      0.000     -27.149     -20.232
x1             0.3112      0